In [1]:
#선반 하나의 객체
import random

class Rack:
    def __init__(self, rack_num, row, col): #(1, 6, 100)으로 선반 객체를 생성하면 '6 X 100 인 1번 Rack' 이 생성
        self.shelf = {}
        self.rack_num = rack_num
        self.row = row
        self.col = col
        for i in range(0,row):
            for j in range(0,col):
                self.shelf[rack_num,i,j] = round(random.random()*15, 2) # 선반 번호, 행, 열 에 해당하는 위치에 15kg 이하 중량 임의 생성

In [2]:
# 토트 박스 하나의 객체
class Tot:
    def __init__(self):
        self.limit_weight = 15 # 토트 박스 내 제품 중량 합의 한계
        self.comp_weight = 0 # 비교하기 위해 사용되는 중량 
        self.now_weight = 0 # 현재 박스 내 제품의 중량 합
        self.tot_dict = {} # 토트의 정보를 저장하기 위한 딕셔너리
    
    # 현재 중량을 반환하는 메서드
    def getNow(self):
        return self.now_weight
    
    # 현재 토트에 새로운 물건의 중량(weight)을 추가할 때, 추가하면 한계를
    # 넘어가는지 안 넘어가는지 판단
    # 넘어가지 않으면 True를 반환, 넘어 간다면 False를 반환
    def fullOrNot(self, weight):
        self.comp_weight = self.now_weight + weight
        if self.limit_weight > self.comp_weight:
            return True
        else:
            return False

In [3]:
# 로봇 객체
class Robot:
    def __init__(self, rack):
        self.rack = rack
        self.horz_time = 0.5  # 가로 이동 시간
        self.start_time = 0.3  # 출발 동작 시간
        self.stop_time = 0.3  # 멈춤 동작 시간
        self.arm_start_time = 0.3  # 피킹 팔 출발 동작 시간
        self.arm_stop_time = 0.3  # 피킹 팔 멈춤 동작 시간
        self.arm_vert_time = 0.5  # 피킹 팔 수직 이동 시간
        self.pick_time = 5  # 제품 추출 시간
        self.drop_time = 5  # 토트 박스 담는 시간
        self.tot_change_time = 5 # 토트 박스 교환 시간
        self.tot_drop_time = 5 # 토트 내보내는 시간
        self.haveTot = self.setTot(20) # 로봇의 초기 빈 토트의 갯수를 세팅
        
    # 초기 토트를 리스트 안에 보관하는 메서드
    def setTot(self, t_num):
        totList = []
        for i in range(0, t_num):
            a = Tot()
            totList.append(a)

        return totList
    
    # 토트 위치는 마지막 행과 동일한 높이라고 가정
    # 따라서 drop 위치는 마지막 행보다 한 칸 위 (row = 6 이고 마지막 칸 index = 5 이므로 drop point = 4)
    def drop_point(self):
        return self.rack.row - 2
    
    # 로봇 팔의 초기 위치를 세팅
    # 로봇 팔의 초기 위치는 전체 행의 갯수의 중앙으로 가정
    # 따라서 초기 위치는 (전체 행의 수)/2, (row = 6, 마지막 칸 indedx = 5 이므로 init point = 5/2 = 2.5)
    def init_point(self):
        return (self.rack.row - 1)/2

In [4]:
# 선반 2개에 로봇 1개의 처리 시뮬레이션
import random

class MRset:
    def __init__(self, p_num, rack1, rack2, picking_product): 
        self.p_num = p_num # picking 해야할 제품의 갯수
        self.rack1 = rack1 # 선반 1
        self.rack2 = rack2 # 선반 2
        self.rackSet = {**self.rack1.shelf, **self.rack2.shelf} # 선반 1,2 를 전체적으로 다루기 위해 지정
        self.robot = Robot(rack1) # 선반 1,2 를 담당할 로봇
        self.picking_product = picking_product # picking 해야할 제품 (위치:중량 형태의 딕셔너리로 전달)
        self.time = self.simulate() # 선반 2개와 로봇 1개의 처리 시간을 가짐
        
    # Simulation about 2 racks used 1 machine
    # 시간의 계산은 본체 이동 시간 + 피킹 마다 걸리는 시간 + 토트 박스 내보내는 시간으로 실시
    def simulate(self):
        time_sec = 0
        time_sec += self.rack1.col * self.robot.horz_time # 한 칸 당 이동 시간이 동일하므로 (열 갯수) * (수평 이동 시간)
        finished_lst = [] # 피킹이 완료된 제품을 담는 리스트
        
        #Robot Point setting
        init_point = self.robot.init_point() # 로봇 팔의 초기 위치는 0-5의 평균값 (중앙 위치) : 2.5
        drop_point = self.robot.drop_point() # 토트의 높이는 한 칸의 높이와 같으므로 drop point 행의 수 - 한 칸 : 5-1 = 4
        
        
        #Tot
        tot_layer = self.robot.haveTot
        tot_num = 1 # 처음 토트의 Number : 1
        now_tot = tot_layer.pop(0) # 토트 목록 중 첫 번째 tot를 선택
        full_tot = {} # 꽉 찬 토트 정보를 저장하는 딕셔너리
        full_tot_layer = [] # 꽉 찬 토트를 저장하는 리스트
        
        #Picking Product
        pick_product = self.picking_product 
        l_pick_product = list(pick_product) # 리스트로 변환해서 피킹해야하는 제품의 위치만을 저장
        
        #보고서 준비를 위한 리스트 변환 및 보고서 시작 (보기 편하게 양식 만들어 출력)
        lrack1 = list(self.rack1.shelf)
        lrack2 = list(self.rack2.shelf) 
        rack_num1 = lrack1.pop(0)[0] 
        rack_num2 = lrack2.pop(0)[0] 
        print("[ rack",rack_num1,"/rack",rack_num2," 결과 보고서 ]\n")
        print("< Picking Product Information > \n")
        print("Picking 목록 : ",pick_product)
        print("Picking 갯수 : ",len(pick_product))
        print("Product Weight 합 : ",sum(pick_product.values()),"\n")
        print("< Tot Box Information > \n")
        
        # 피킹하는 작업에 대한 시간 계산
        while len(finished_lst) < self.p_num:
            
            curTot = now_tot # 현재 토트 = 사용 중인 토트
            now_product = l_pick_product.pop(0) # 피킹하고자 하는 제품의 위치 정보의 제일 앞에서 pop 
            curProd_weight = self.rackSet[now_product] # 피킹하고자 하는 제품의 중량을 rackSet에서 뽑아서 저장
            
            # 마지막 위치 기반 시간 계산
            cur_point_y = now_product[1]
            
            time_sec += (self.robot.start_time
                     + self.robot.stop_time
                     + self.robot.arm_start_time        # 로봇 팔 출발
                     + (self.robot.arm_vert_time  
                     * abs(init_point - cur_point_y))   # | 초기 위치 - 현재 위치 | * 팔 수직 이동 시간
                     + self.robot.arm_stop_time         # 로봇 팔 멈춤
                     + self.robot.pick_time             # 순수 피킹 시간
                     + self.robot.arm_start_time        # 피킹 후 로봇 팔 출발
                     + (self.robot.arm_vert_time
                     * abs(drop_point - cur_point_y))   # | 현재 위치 - 토트 박스 위치 | * 팔 수직 이동 시간
                     + self.robot.arm_stop_time         # 토트 박스 위치에서 팔 멈춤
                     + self.robot.drop_time             # 토트 박스에 넣는 시간
                     + self.robot.arm_start_time        # 토트에 넣은 후 출발 
                     + (self.robot.arm_vert_time
                     * abs(drop_point - init_point))    # | 토트박스 위치 - 초기 위치 | * 팔 수직 이동 시간
                     + self.robot.arm_stop_time         # 초기 위치에서 팔 멈춤
                     )
            
            finished_lst.append(now_product) # 피킹이 끝난 제품을 확인하기 위한 리스트
            
            if curTot.fullOrNot(curProd_weight):                # 토트가 가득 찼는지를 판단하는 메서드 (가득차지 않았으면 True)
                curTot.tot_dict[now_product] = curProd_weight   # 피킹한 제품의 위치와 중량을 저장하는 딕셔너리
                curTot.now_weight += curProd_weight             #  토트내 중량 최신화
                
            else: # 토트가 가득 찼을 때
                full_tot[tot_num] = now_tot.now_weight 
                full_tot_layer.append(now_tot)                  # 가득 찬 토트를 저장하는 딕셔너리에 토트 번호와 중량 저장
                print(tot_num,"번 토트 : ", now_tot.tot_dict)   # 토트가 가득 찼을 때 토트 정보 출력
                tot_num += 1                                    # 토트 번호 업그레이드
                now_tot = tot_layer.pop(0)                      # 현재 토트를 토트 레이어에서 다음 토트로 최신화
                time_sec += self.robot.tot_change_time          # 토트 교환 시간
                curTot = now_tot # 현재 토트 최신화
                curTot.tot_dict[now_product] = curProd_weight
                curTot.now_weight += curProd_weight             # 최신화된 토트에 중량 추가
            
            
                
        full_tot[tot_num] = now_tot.now_weight # 마지막 토트가 가득 차지 않았더라도 작업이 끝났으므로 토트 딕셔너리에 저장
        time_sec += self.robot.tot_drop_time * len(full_tot) # 토트 하나당 내보내는 시간 X 토트 갯수
        
        for elmt in finished_lst: # pciking한 위치에 대해 중량을 0으로 바꿔 줌
            if elmt[0]%2 == 1 : self.rack1.shelf[elmt] = 0
            elif elmt[0]%2 == 0: self.rack2.shelf[elmt] = 0 
        
        # 결과 보고서 형식으로 프린트
        print(tot_num,"번 토트는", now_tot.tot_dict)
        print("토트 정보 : ", full_tot)
        print("토트 박스 수 : ", len(full_tot), "\n")
        print("__라인 처리 시간__ ")
        self.hms(time_sec) 

        # 피킹이 정확한 위치에 됐는지 확인하는 보고서
        check_dict = {}
        for position in finished_lst:
            if position[0]%2 == 1: check_dict[position] = self.rack1.shelf[position]
            elif position[0]%2 == 0: check_dict[position] = self.rack2.shelf[position]
        print("\n 빈 위치 : ",check_dict)   
        print("="*125,"\n")
        
        return time_sec
    
    # 초 단위를 시, 분, 초로 분할
    def hms(self, time):
        hour = time // 3600
        time = time - hour * 3600
        mi = time // 60
        time = time - mi * 60
        print(hour, "시간 ", mi, "분 ", time, "초")

In [5]:
# 1피킹할 제품을 랜덤으로 생성
def prob_Product(num, setOfRack): # (피킹하고자 하는 제품 갯수, 선반 - 딕셔너리 형태로 전달)
    lrackSet = list(setOfRack)
    l_prod = []

    # 피킹하고자 하는 제품 갯수를 랜덤으로 피킹
    while len(l_prod) < num:
        rlst = random.choice(lrackSet)
        if not rlst in l_prod:
            l_prod.append(rlst)

    # 로봇의 작동 순서에 따라 정렬
    reverse_l_prod = reverseTup(l_prod)
    reverse_l_prod.sort()    
    result = reverseTup(reverse_l_prod)


    d_prod = {}
    # 딕셔너리 형태로 재생성
    while len(d_prod) < num:
        keyy = result.pop(0)
        d_prod[keyy] = setOfRack[keyy]

    return d_prod

# 리스트 반대 정렬을 위해 사용되는 메서드
# 피킹 시 로봇이 열이 커지는 방향으로 이동하므로 열, 행, 선반 번호 순으로 정렬해 줌
def reverseTup(lst):
        reverse_lst = []
        for elmt in lst:
            l_elmt = list(elmt)
            tmp = l_elmt[0]
            l_elmt[0] = l_elmt[2] 
            l_elmt[2] = tmp
            t_elmt = tuple(l_elmt)
            reverse_lst.append(t_elmt)
        return reverse_lst

In [6]:
# 10개의 선반 생성
r1 = Rack(1, 6, 100)
r2 = Rack(2, 6, 100)
r3 = Rack(3, 6, 100)
r4 = Rack(4, 6, 100)
r5 = Rack(5, 6, 100)
r6 = Rack(6, 6, 100)
r7 = Rack(7, 6, 100)
r8 = Rack(8, 6, 100)
r9 = Rack(9, 6, 100)
r10 = Rack(10, 6, 100)

# 저장소 내의 모든 선반
allOfRack = {**r1.shelf,**r2.shelf,**r3.shelf,**r4.shelf,**r5.shelf
            ,**r6.shelf,**r7.shelf,**r8.shelf,**r9.shelf,**r10.shelf
            }

# 저장소 내에서 50개의 피킹할 제품을 임의 생성
product = prob_Product(50, allOfRack)

In [7]:
# 각 선반에 할당된 제품을 나누는 메서드
# 예를 들어 1,2 번 선반에는 선반 번호가 1,2에 해당하는 피킹 제품들이 전달
def division_product(rack_num, product):
    list_of_product = list(product)
    list_of_product.sort()
    list_of_dvsn_product = {}
    prod_num = len(product)
    
    for i in range(1,10,2):
        add_to_product = {}
        for elmt in list_of_product:
            if elmt[0] == i or elmt[0] == i+1:
                add_to_product[elmt] = product[elmt]
                
        reverse_l_prod = reverseTup(list(add_to_product))
        reverse_l_prod.sort()    
        result = reverseTup(reverse_l_prod)


        d_prod = {}

        while len(result) > 0:
            keyy = result.pop(0)
            d_prod[keyy] = product[keyy]
            
        list_of_dvsn_product[i,i+1] = d_prod
        
    return list_of_dvsn_product
        
dv_prod = division_product(10, product)

In [8]:
# (선반 한 쌍에 전달되는 피킹 제품 수, 선반1, 선반2, 선반 한 쌍에서 피킹되어야 하는 제품 딕셔너리) 형태로 simulation 진행
# 즉, 선반 2개 로봇 1개에 대한 시뮬레이션을 5세트 실시
r12 = MRset(len(dv_prod[1,2]), r1, r2, dv_prod[1,2])
r34 = MRset(len(dv_prod[3,4]), r3, r4, dv_prod[3,4])
r56 = MRset(len(dv_prod[5,6]), r5, r6, dv_prod[5,6])
r78 = MRset(len(dv_prod[7,8]), r7, r8, dv_prod[7,8])
r910 = MRset(len(dv_prod[9,10]), r9, r10, dv_prod[9,10])

# 10개 선반에 대해 시스템 종료 시간 출력 (가장 늦게 끝나는 set이 시스템 종료 시간이므로 가장 오래 걸리는 선반과 시간을 따로 출력

max_line = ()
d_time = {r12:r12.time, r34:r34.time, r56:r56.time, r78:r78.time, r910:r910.time}
late_time = max(r12.time, r34.time, r56.time, r78.time, r910.time)

for key in d_time.keys():
    if d_time[key] == late_time: max_line = key
        
print("해당 라인 : ",max_line.rack1.rack_num,",",max_line.rack2.rack_num)
print("\n__시스템 종료 시간__")
max_line.hms(late_time)

[ rack 1 /rack 2  결과 보고서 ]

< Picking Product Information > 

Picking 목록 :  {(2, 1, 5): 4.0, (1, 5, 21): 14.8, (1, 2, 37): 6.48, (1, 3, 42): 10.3, (2, 2, 48): 14.62}
Picking 갯수 :  5
Product Weight 합 :  50.199999999999996 

< Tot Box Information > 

1 번 토트 :  {(2, 1, 5): 4.0}
2 번 토트 :  {(1, 5, 21): 14.8}
3 번 토트 :  {(1, 2, 37): 6.48}
4 번 토트 :  {(1, 3, 42): 10.3}
5 번 토트는 {(2, 2, 48): 14.62}
토트 정보 :  {1: 4.0, 2: 14.8, 3: 6.48, 4: 10.3, 5: 14.62}
토트 박스 수 :  5 

__라인 처리 시간__ 
0.0 시간  2.0 분  48.00000000000003 초

 빈 위치 :  {(2, 1, 5): 0, (1, 5, 21): 0, (1, 2, 37): 0, (1, 3, 42): 0, (2, 2, 48): 0}

[ rack 3 /rack 4  결과 보고서 ]

< Picking Product Information > 

Picking 목록 :  {(3, 1, 2): 11.18, (4, 4, 44): 3.93, (4, 5, 48): 4.21, (3, 2, 68): 4.39, (4, 3, 68): 8.32, (3, 0, 83): 14.4, (4, 1, 96): 14.75}
Picking 갯수 :  7
Product Weight 합 :  61.18 

< Tot Box Information > 

1 번 토트 :  {(3, 1, 2): 11.18}
2 번 토트 :  {(4, 4, 44): 3.93, (4, 5, 48): 4.21, (3, 2, 68): 4.39}
3 번 토트 :  {(4, 3, 68): 8.32}
4 번 토트 